In [1]:
# import dependencies
import pandas as pd
import datetime
from dateutil import parser
import requests
import json
import random as ran
import numpy as np


In [2]:
OMs = pd.read_csv('TCB_mics_calendar - TCB_mics_calendar copy 6.csv')

OMs = OMs.dropna(subset=['EVENT NAME', 'VENUE'])

columns = {}
drops = []

for col in OMs.columns:
    c = col.replace(' - MIC', '').replace('CATEGORIES', 'CATEGORY').replace('?', '').replace(' MICS', '').replace(' ', '_').lower()
    if '(dont_fill)' in c:
        drops.append(c)
    columns[col] = c
    
OMs = OMs.rename(columns=columns)

OMs = OMs.drop(columns=drops)

OMs




,event_name,venue,neighborhood,start_date,start_time,end_date,end_time,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,event_description,stage_time,event_website
0,(11AM-12PM Hourly) Improv Mic @ Fourth Wall Co...,Fourth Wall Comedy Cafe,Hollywood/East Hollywood,2020-03-15,11:00 AM,2020-03-15,1:45 PM,False,Open Mic,$5,No min.,NaN,True,True,For stand-up riffing and crowd work\nSuggestio...,5 min.,https://slotted.co/fwcafe
1,Improv Mic,The Hollywood Comedy,Hollywood/East Hollywood,2020-03-15,11:00 AM,2020-03-15,12:00 PM,False,Improvised stand-up mic,$5,No min.,NaN,True,True,includes snacks and drinks\nSign-up in advance...,5 min.,https://slotted.co/thehollywoodcomedy
2,(1PM-6PM Hourly) The Hollywood Comedy,The Hollywood Comedy,Hollywood/East Hollywood,2020-03-08,1:00 PM,2020-03-08,7:00 PM,False,Open Mic,$5,No min.,NaN,True,True,includes snacks & drinks\nSign-up in advance a...,5 min.,https://slotted.co/thehollywoodcomedy
3,(12PM-1PM Hourly) Flashback Mic (in Melrose),Flashback Mic (on Melrose),WeHo/Fairfax/Beverly Hills,2020-03-15,12:00 PM,2020-03-15,2:00 PM,False,Open Mic,$5,No min.,NaN,True,True,"*entrance located in rear, look for the red do...",5 min.,https://slotted.co/fbmic
4,(1PM-12AM Hourly) 15 Mins. The Hollywood Comedy,The Hollywood Comedy,Hollywood/East Hollywood,2020-03-08,1:00 PM,2020-03-08,11:59 PM,False,Open Mic,$12,No min.,NaN,True,True,"\n\nin ""The Fancy Room"" (up front)\n$12 (inclu...",15 min.,https://slotted.co/thehollywoodcomedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,Geeky Teas & Games,Geeky Teas & Games,Burbank/NoHo/Valley,2020-02-22,9:30 PM,2020-02-22,11:30 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes bottled water\nSign-up 9PM\nStart...,5 min.,NaN
457,Santa Monica Playhouse,Santa Monica Playhouse,Westside/Santa Monica,2020-02-29,9:30 PM,2020-02-29,12:00 AM,False,Open Mic,$5,No min.,NaN,True,True,"\n\nNot on Main Stage, in Other Theatre\nPassw...",5 min.,NaN
458,Echoes on Pico,Echoes on Pico,Koreatown/Mid-City,2020-03-14,10:00 PM,2020-03-14,11:00 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nincludes drink\nSign-up in advance at http...,5 min.,https://slotted.co/oxo0opzu/
459,Atomic Wombat Comics,Atomic Wombat Comics,Orange County,2020-03-07,10:30 PM,2020-03-07,11:30 PM,False,Open Mic,$5,No min.,NaN,True,True,\n\nSign-up in advance at https://slotted.co/c...,5 min.,https://slotted.co/comedygrabbag


In [3]:
# convert to timestamps

begin_and_end = ['start_date', 'end_date', 'start_time', 'end_time']

for i , t in enumerate(begin_and_end):
    if i < 2:
        OMs[t] = OMs[t].apply(lambda x: parser.parse(x))
    else:
        OMs[t] = OMs[t].apply(lambda x: parser.parse(x).hour)



In [4]:
events = OMs['event_name'].unique()


In [5]:
OMs = OMs.sort_values(['start_date', 'start_time'])

In [6]:

for e in events:
    
    # all instances of an event

    event = OMs.copy().loc[OMs['event_name'] == e,:]

    rows = list(event.index)

    # consolidate times
    new = 0
    start = event['start_time'][rows[0]]
    for i in range(1, len(rows)):
        end = event[f'end_time'][rows[i]]
        if event['end_time'][rows[i-1]] == event['start_time'][rows[i]]:
            OMs.loc[rows[i], 'start_time'] = start
            OMs.loc[rows[new:i], 'end_time'] = end
        else: 
            new = i
            start = event['start_time'][rows[i]]

    # consolidate dates
    new = 0
    start = event['start_date'][rows[0]]
    for i in range(1, len(rows)):
        end = event[f'end_date'][rows[i]]
        if (event['start_date'][rows[i]] - event['end_date'][rows[i-1]]).days < 2:
            OMs.loc[rows[i], 'start_date'] = start
            OMs.loc[rows[new:i], 'end_date'] = end
        else:
            new = i
            start = event['start_date'][rows[i]]



In [7]:
OMs = OMs.drop(columns='event_description')

len(OMs)

454

In [8]:
OMs = OMs.drop_duplicates().reset_index(drop=True)

In [9]:
OMs

,event_name,venue,neighborhood,start_date,start_time,end_date,end_time,all_day_event,category,event_cost,purchase_req,free,show_map_link,show_map,stage_time,event_website
0,Storytelling Mic,LA Wine,DTLA,2020-01-13,19,2020-01-13,21,False,Storytelling,$0,No min.,Free mics\n,True,True,5 min.,NaN
1,LA Connection Comedy Theater,LA Connection Comedy Theater,Burbank/NoHo/Valley,2020-01-16,18,2020-01-16,19,False,Open Mic,$5,No min.,NaN,True,True,5 min.,https://slotted.co/hotmedusa-
2,The Barkley,The Barkley,Pasadena,2020-01-22,21,2020-01-22,23,False,Open Mic,$0,No min.,Free mics\n,True,True,3 min.,NaN
3,(7-9PM Hourly) The Open Mic Spot,The Open Mic Spot,South LA,2020-01-24,19,2020-01-24,21,False,Open Mic,$5,No min.,NaN,True,True,5 min.,https://openmicspot.com/open-mic/
4,(8-10PM Hourly) Therapeutic Noise (in Long Beach),Therapeutic Noise,Long Beach,2020-01-25,20,2020-01-25,23,False,Open Mic,$5,No min.,NaN,True,True,6 min.,https://slotted.co/therapeuticnoise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,Edendale Branch Library,Edendale Branch Library,Los Feliz/Silverlake/Echo Park,2020-04-02,18,2020-04-02,20,False,Open Mic,$0,No min.,Free mics\n,True,True,5 min.,NaN
259,Improv Jam,UCB Sunset Inner Sanctum,Hollywood/East Hollywood,2020-04-03,18,2020-04-03,19,False,Open Mic,$0,No min.,Free mics\n,True,True,NaN,NaN
260,Lyric Hyperion Theatre & Cafe,Lyric Hyperion Theatre & Cafe,Los Feliz/Silverlake/Echo Park,2020-04-04,16,2020-04-04,18,False,Open Mic,NaN,1 item min.,NaN,True,True,4 min.,NaN
261,Character Spa,Lyric Hyperion Theatre & Cafe,Los Feliz/Silverlake/Echo Park,2020-04-06,16,2020-04-06,18,False,Character Mic,$0,No min.,Free mics\n,True,True,7 min.,https://slotted.co/characterspa
